In [1]:
from falkordb_gemini_kg import (
    Orchestrator,
    Ontology,
    OrchestratorRunner,
    Entity,
    Relation,
    Attribute,
    AttributeType,
    KnowledgeGraph,
    KnowledgeGraphModelConfig,
)
from falkordb_gemini_kg.agents.kg_agent import KGAgent
from falkordb_gemini_kg.models.gemini import GeminiGenerativeModel

from json import loads
from falkordb import FalkorDB
import os
import vertexai
import logging

logging.basicConfig(level=logging.DEBUG)

vertexai.init(project=os.getenv("PROJECT_ID"), location=os.getenv("REGION"))

restaurants_ontology = Ontology()
restaurants_ontology.add_entity(
    Entity(
        label="Country",
        attributes=[
            Attribute(
                name="name",
                attr_type=AttributeType.STRING,
                required=True,
                unique=True,
            ),
        ],
    )
)
restaurants_ontology.add_entity(
    Entity(
        label="City",
        attributes=[
            Attribute(
                name="name",
                attr_type=AttributeType.STRING,
                required=True,
                unique=True,
            ),
            Attribute(
                name="weather",
                attr_type=AttributeType.STRING,
                required=False,
                unique=False,
            ),
            Attribute(
                name="population",
                attr_type=AttributeType.NUMBER,
                required=False,
                unique=False,
            ),
        ],
    )
)
restaurants_ontology.add_entity(
    Entity(
        label="Restaurant",
        attributes=[
            Attribute(
                name="name",
                attr_type=AttributeType.STRING,
                required=True,
                unique=True,
            ),
            Attribute(
                name="description",
                attr_type=AttributeType.STRING,
                required=False,
                unique=False,
            ),
            Attribute(
                name="rating",
                attr_type=AttributeType.NUMBER,
                required=False,
                unique=False,
            ),
            Attribute(
                name="food_type",
                attr_type=AttributeType.STRING,
                required=False,
                unique=False,
            ),
        ],
    )
)
restaurants_ontology.add_relation(
    Relation(
        label="IN_COUNTRY",
        source="City",
        target="Country",
    )
)
restaurants_ontology.add_relation(
    Relation(
        label="IN_CITY",
        source="Restaurant",
        target="City",
    )
)


attractions_ontology = Ontology()
attractions_ontology.add_entity(
    Entity(
        label="Country",
        attributes=[
            Attribute(
                name="name",
                attr_type=AttributeType.STRING,
                required=True,
                unique=True,
            ),
        ],
    )
)
attractions_ontology.add_entity(
    Entity(
        label="City",
        attributes=[
            Attribute(
                name="name",
                attr_type=AttributeType.STRING,
                required=True,
                unique=True,
            ),
            Attribute(
                name="weather",
                attr_type=AttributeType.STRING,
                required=False,
                unique=False,
            ),
            Attribute(
                name="population",
                attr_type=AttributeType.NUMBER,
                required=False,
                unique=False,
            ),
        ],
    )
)
attractions_ontology.add_entity(
    Entity(
        label="Attraction",
        attributes=[
            Attribute(
                name="name",
                attr_type=AttributeType.STRING,
                required=True,
                unique=True,
            ),
            Attribute(
                name="description",
                attr_type=AttributeType.STRING,
                required=False,
                unique=False,
            ),
            Attribute(
                name="type",
                attr_type=AttributeType.STRING,
                required=False,
                unique=False,
            ),
        ],
    )
)
attractions_ontology.add_relation(
    Relation(
        label="IN_COUNTRY",
        source="City",
        target="Country",
    )
)
attractions_ontology.add_relation(
    Relation(
        label="IN_CITY",
        source="Attraction",
        target="City",
    )
)


model = GeminiGenerativeModel("gemini-1.5-flash-001")
restaurants_kg = KnowledgeGraph(
    name="restaurants",
    ontology=restaurants_ontology,
    model_config=KnowledgeGraphModelConfig.with_model(model),
)
attractions_kg = KnowledgeGraph(
    name="attractions",
    ontology=attractions_ontology,
    model_config=KnowledgeGraphModelConfig.with_model(model),
)


def import_data():
    cities = loads(open("data/cities.json").read())
    restaurants = loads(open("data/restaurants.json").read())
    attractions = loads(open("data/attractions.json").read())

    for city in cities:
        restaurants_kg.add_node(
            "City",
            {
                "name": city["name"],
                "weather": city["weather"],
                "population": city["population"],
            },
        )
        restaurants_kg.add_node("Country", {"name": city["country"]})
        restaurants_kg.add_edge(
            "IN_COUNTRY",
            "City",
            "Country",
            {"name": city["name"]},
            {"name": city["country"]},
        )

        attractions_kg.add_node(
            "City",
            {
                "name": city["name"],
                "weather": city["weather"],
                "population": city["population"],
            },
        )
        attractions_kg.add_node("Country", {"name": city["country"]})
        attractions_kg.add_edge(
            "IN_COUNTRY",
            "City",
            "Country",
            {"name": city["name"]},
            {"name": city["country"]},
        )

    for restaurant in restaurants:
        restaurants_kg.add_node(
            "Restaurant",
            {
                "name": restaurant["name"],
                "description": restaurant["description"],
                "rating": restaurant["rating"],
                "food_type": restaurant["food_type"],
            },
        )
        restaurants_kg.add_edge(
            "IN_CITY",
            "Restaurant",
            "City",
            {"name": restaurant["name"]},
            {"name": restaurant["city"]},
        )

    for attraction in attractions:
        attractions_kg.add_node(
            "Attraction",
            {
                "name": attraction["name"],
                "description": attraction["description"],
                "type": attraction["type"],
            },
        )
        attractions_kg.add_edge(
            "IN_CITY",
            "Attraction",
            "City",
            {"name": attraction["name"]},
            {"name": attraction["city"]},
        )

In [2]:
import_data()

In [2]:
restaurants_agent = KGAgent(
    agent_id="restaurants_agent",
    kg=restaurants_kg,
    introduction="""
I'm a restaurant agent, specialized in finding the best restaurants for you. 
""",
)

attractions_agent = KGAgent(
    agent_id="attractions_agent",
    kg=attractions_kg,
    introduction="""
I'm an attractions agent, specialized in finding the best attractions for you. 
""",
)

orchestrator = Orchestrator(model)

orchestrator.register_agent(restaurants_agent)
orchestrator.register_agent(attractions_agent)

runner = orchestrator.ask("Write me a 3 day itinerary for a trip to Italy")

DEBUG:google.auth.transport.requests:Making request: POST https://oauth2.googleapis.com/token
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): oauth2.googleapis.com:443
DEBUG:urllib3.connectionpool:https://oauth2.googleapis.com:443 "POST /token HTTP/1.1" 200 None
DEBUG:falkordb_gemini_kg.classes.orchestrator:Execution plan response: ```json
[{"id": "1", "block": "parallel", "properties": {"steps": [{"id": "1.1", "block": "prompt_agent", "properties": {"agent": "attractions_agent", "prompt": "Suggest 3 attractions for a 3 day trip to Italy."}}, {"id": "1.2", "block": "prompt_agent", "properties": {"agent": "restaurants_agent", "prompt": "Suggest 3 restaurants in Italy."}}]}}, {"id": "2", "block": "summary"}]
```
DEBUG:falkordb_gemini_kg.classes.orchestrator:Execution plan: ExecutionPlan(steps={'steps': [{'id': '1', 'block': 'parallel', 'properties': {'steps': [{'id': '1.1', 'block': 'prompt_agent', 'properties': {'agent': 'attractions_agent', 'prompt': 'Suggest 3 attracti

In [3]:
response = runner.run()
print(response.text)

DEBUG:falkordb_gemini_kg.steps.graph_query_step:Cypher Prompt: 
Using the ontology provided, generate an OpenCypher statement to query the graph database returning all relevant entities, relationships, and attributes to answer the question below:
If you cannot generate a OpenCypher statement for any reason, return an empty string.
Respect the order of the relationships, the arrows should always point from the "source" to the "target".

Question: Provide a list of top attractions in Italy for a 3 day itinerary.

DEBUG:falkordb_gemini_kg.steps.graph_query_step:Cypher Prompt: 
Using the ontology provided, generate an OpenCypher statement to query the graph database returning all relevant entities, relationships, and attributes to answer the question below:
If you cannot generate a OpenCypher statement for any reason, return an empty string.
Respect the order of the relationships, the arrows should always point from the "source" to the "target".

Question: Suggest some restaurants in Italy

## 3-Day Itinerary for Italy:

**Day 1:**

* **Morning:** Visit the Colosseum in Rome, followed by a tour of the Vatican Museums.
* **Afternoon:** Enjoy a leisurely lunch at Trattoria da Enzo in Rome, then explore the Trevi Fountain.
* **Evening:** Have dinner at Il Pagliaccio in Rome and stroll through the vibrant Piazza Navona.

**Day 2:**

* **Morning:** Take a high-speed train to Milan and visit the Galleria Vittorio Emanuele II.
* **Afternoon:**  Have lunch at Bros' Burger in Milan and visit the Teatro alla Scala.
* **Evening:** Enjoy a delicious dinner at Il Gambero Rosso in Naples and experience the city's lively nightlife.

**Day 3:**

* **Morning:** Explore the historical center of Palermo, visiting the Cattedrale di Palermo and the Palazzo dei Normanni.
* **Afternoon:** Enjoy a traditional Sicilian lunch at Da Cimino in Palermo.
* **Evening:** Take a leisurely stroll through the historic district of Venice, admiring the Doge's Palace, St. Mark's Basilica, and the Rialto Bridg